# Ex7 - Descrição e transformações de modelos

Nesta atividade, vocês vão exercitar os conceitos de descrição de um modelo e de transformações geométricas. Para evitar erros de execução, utilize apenas uma célula de código para cada parte desta atividade.

### Parte 1 - Criando uma malha de triângulos a partir de uma equação 

Complete a definição da função create_circle(fatias, raio) para criar uma malha triangular de um modelo circular centrado na origem a partir da equação paramétrica do círculo. O parâmetro 'fatias' é o número de divisões do modelo.

$x = raio * \cos (angle)$

$y = raio * \sin (angle)$

Os vértices podem ser definidos de duas maneiras. A primeira é utilizando apenas um vetor contendo as posições dos vértices dos triângulos. E a outra é utilizando dois vetores, um contendo as posições dos vértices e o outro contendo os índices dos vértices dos triângulos. Perceba que o recurso de culling do OpenGL está ativado. Então, a orientação dos vértices dos triângulos deve ser no sentido anti-horário. Você pode usar como base o notebook ([25_Formas_de_especificar_um_modelo](25_Formas_de_especificar_um_modelo.ipynb))

![title](cg/images/ex7_image.png)

In [ ]:
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer

def create_circle(fatias, raio):

#     return vertex_position
#     return return vertex_position, vertex_indices

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        # cria a malha de triângulos 
#         circle_mesh_positions = create_circle(25, 0.7)
#         circle_mesh_positions, circle_mesh_indices = create_circle(25, 0.7)
        
        #cria um objeto responsável por carregar os dados para a GPU e renderizá-los
#         self.circleRenderer = ModelRenderer(circle_mesh_positions)
#         self.circleRenderer = ModelRenderer(circle_mesh_positions, vertex_indices=circle_mesh_indices)
        

        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
        
        # ativa o shader programa para configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # recupera o endereços da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.circleRenderer.setVertexPositionLoc(position_loc)
        
    def paintGL(self):
        # configura a cor de background
        gl.glClearColor(0, 0, 0, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT)
        
        # ativa o recurso de Culling
        gl.glEnable(gl.GL_CULL_FACE)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # configura a cor do círculo
        self.shaderProgram.setUniformColor(np.array([0.0, 0.0, 0.5, 1.0], dtype=np.float32))
        
        # renderiza o círculo
        self.circleRenderer.render()
        
        # configura a cor do wireframe
        self.shaderProgram.setUniformColor(np.array([1.0, 1.0, 1.0, 1.0], dtype=np.float32))
        
        # renderiza o wireframe
        self.circleRenderer.renderWireframe()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()

    def resizeGL(self, width, height):
        
        # atualiza a área de renderização para ser a janela inteira
        gl.glViewport(0, 0, width, height)

def main():
    import sys

    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    w = MyWidget(glformat)
    w.resize(640, 480)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

### Parte 2 - Quadrados loucos

Crie um programa que realize a animação descrita na imagem abaixo. Nela, há dois quadrados com as posições fixas no eixo x (fora da origem) e que giram no próprio eixo. Há também dois quadrados girando em torno da origem, mas em direções opostas. Cada triângulo possui uma cor diferente. Além disso, utilize os eventos da tecla + (QtCore.Qt.Key_Plus) e - (QtCore.Qt.Key_Minus) para, respectivamente, acelerar e diminuir a velocidade da rotação dos objetos. Você pode usar como base os notebooks ([27_Quadrado_com_transformacoes](27_Quadrado_com_transformacoes.ipynb)) e ([24_Eventos_do_teclado](24_Eventos_do_teclado.ipynb)).

![title](cg/images/ex7_squares.png)

### Parte 3 - Posicionando a lua

Complete a definição das funções 'renderMoonOrbit()' e 'renderMoon()' do código abaixo para renderizar dois objetos. O primeiro objeto é a orbita da lua acompanhando a movimentação da Terra. O segundo objeto é a lua girando em torno da Terra. Também é preciso criar na função 'initializeGL()' duas variáveis (semelhantes às variáveis 'self.earthOrbit' e 'self.earthPlanet') responsáveis por armazenar as informações gráficas da lua e da sua órbita. A animação final deve ser semelhante à imagem abaixo.

Você pode usar como base o notebook ([27_Quadrado_com_transformacoes](27_Quadrado_com_transformacoes.ipynb)).

![title](cg/images/ex7_earth_moon.png)

In [1]:
import glm
import time
import math
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL, QtCore
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v2 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v2 import ModelRenderer
from cg.models.SphereMesh_v1 import SphereMesh
from cg.models.OrbitPolygon import OrbitPolygon

AU = 1.496E8
planet_scale_factor = 1500.0
moon_scale_factor = planet_scale_factor * 0.05
sun_scale_factor = 50

# classe para armazenar as informações do Sol
class SunInfo:
    def __init__(self, color, diameter):
        
        self.color = color
        self.diameter = (diameter / AU) * sun_scale_factor
        self.radius = self.diameter / 2.0

# classe para armazenar as informações dos planetas
class PlanetInfo:
    def __init__(self, color, diameter, orbitalPeriod, orbitalInclination, orbitSemiMajorAxis, orbitSemiMinorAxis, orbitFocus):
        
        earth_days = 365.2
        
        self.color = color
        self.diameter = (diameter / AU) * planet_scale_factor
        self.radius = self.diameter / 2.0
        self.velocity = earth_days / orbitalPeriod
        
        self.orbitalPeriod = orbitalPeriod
        self.orbitalInclination = orbitalInclination 
        self.orbitSemiMajorAxis = orbitSemiMajorAxis
        self.orbitSemiMinorAxis = orbitSemiMinorAxis
        self.orbitFocus = orbitFocus

# cria um dicionário contendo as informações do Sol, da Terra e da lua
PlanetarySheet = {
       'sun': SunInfo(np.array([1.00, 1.00, 0.00, 1.0]), 1391900),
       'earth': PlanetInfo(np.array([0.61, 0.79, 0.37, 1.0]), 12742, 365.2, 1.57, 1.0027, 1.0025, 0.0167),
       'moon': PlanetInfo(np.array([0.50, 0.50, 0.50]), 3475, 27.3, 5.1, 0.0025718 * moon_scale_factor, 0.0021479 * moon_scale_factor, 0.00014537)
       }

# classe para armazenar as informações gráficas em um único objeto
class GraphicObject:
    def __init__(self):
        self.model = None
        self.renderer = None
        self.modelMatrix = glm.mat4()
        self.color = np.array([1.0, 1.0, 1.0, 1.0])
        
class MyWidget(QtOpenGL.QGLWidget):

    def initializeGL(self):
        
        # define uma transformação global
        # os objetos no tamanho original não cabem no volume de renderização. 
        # Então eles precisam ser escalado para um tamanho menor
        self.globalTransform = glm.scale(glm.mat4(), glm.vec3(0.8, 0.8, 0.8))
        
        # cria um shader program simples     
        self.shaderProgram = SimpleShaderProgram()
        
        # recupera o endereços da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        
        # ativa o shader programa para permitir configurar uma cor única para todos os vértices
        self.shaderProgram.bind()
        self.shaderProgram.useUniformColor(True)
        self.shaderProgram.release()

        # cria uma malhar de triângulos esférica comum para o Sol, a Terra e a lua
        sphere_mesh = SphereMesh(1.0, 30, 30)
        sphere_renderer = ModelRenderer(sphere_mesh.getVertexPositions(), vertex_indices=sphere_mesh.getVertexIndices())
        sphere_renderer.setVertexPositionLoc(position_loc)
        
        # cria um objeto para armazenar as informações gráficas da órbita da Terra
        planet_info = PlanetarySheet['earth']
        self.earthOrbit = GraphicObject()
        self.earthOrbit.model = OrbitPolygon(planet_info.orbitSemiMajorAxis, planet_info.orbitSemiMinorAxis, planet_info.orbitFocus, 50)
        self.earthOrbit.renderer = ModelRenderer(self.earthOrbit.model.getVertexPositions(), primitive=ModelRenderer.LINE_LOOP)
        self.earthOrbit.renderer.setVertexPositionLoc(position_loc)
        self.earthOrbit.color = planet_info.color
        self.earthOrbit.modelMatrix = self.globalTransform 
        
        # cria um objeto para armazenar as informações gráficas do planeta Terra
        self.earthPlanet = GraphicObject()
        self.earthPlanet.model = sphere_mesh
        self.earthPlanet.renderer = sphere_renderer
        self.earthPlanet.color = PlanetarySheet['earth'].color
        
        # cria um objeto para armazenar as informações gráficas do Sol
        self.sun = GraphicObject()
        self.sun.model = sphere_mesh
        self.sun.renderer = sphere_renderer
        self.sun.color = PlanetarySheet['sun'].color
        self.sun.modelMatrix = glm.scale(glm.mat4(), glm.vec3(PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius, PlanetarySheet['sun'].radius))
        
        # armazena o momento que o programa começou
        self.startTime = time.time()
        
        ####################################################################################
        # Adicione aqui o código de criação dos objetos que representam a lua e sua órbita # 
        ####################################################################################
    
    def paintGL(self):
        
        # configura a cor de background
        gl.glClearColor(1, 1, 1, 1)
        
        # limpa o background com a cor especificada
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o teste de profunidade
        gl.glEnable(gl.GL_DEPTH_TEST);
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_elapsed = self.currentTime - self.startTime
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()

        # renderiza os objetos 
        self.renderSun()
        self.renderEarthOrbit()
        self.renderEarthPlanet(time_elapsed)
        self.renderMoonOrbit(time_elapsed)
        self.renderMoon(time_elapsed)
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def resizeGL(self, width, height):
        
        # atualiza a área de renderização para ser a janela inteira
        gl.glViewport(0, 0, width, height)
        
    def renderSun(self):
    
        # atualiza o shader com a transformação e a cor do Sol
        self.shaderProgram.setUniformMVPMatrix(self.sun.modelMatrix)
        self.shaderProgram.setUniformColor(self.sun.color)
        
        # renderiza o Sol
        self.sun.renderer.render()
    
    def renderEarthOrbit(self):
        
        # atualiza o shader com a transformação e a cor da órbita da Terra
        self.shaderProgram.setUniformMVPMatrix(self.earthOrbit.modelMatrix)
        self.shaderProgram.setUniformColor(self.earthOrbit.color)
        
        # renderiza a órbita da Terra
        self.earthOrbit.renderer.render()
        
    def renderEarthPlanet(self, time_elapsed):
        
        angle = time_elapsed * 2 * math.pi * 0.05
        
        # recupera a posição da Terra na órbita
        planet_info = PlanetarySheet['earth']
        x, y = self.earthOrbit.model.getPositionAt(angle * planet_info.velocity)
        
        # cria uma matriz de translação para levar a Terra para a nova posição
        transl_planet_pos = glm.translate(glm.mat4(), glm.vec3(x, y, 0.0))
        
        # cria uma matriz de escala para deixar a Terra com o tamanho correto
        scale_planet_size = glm.scale(glm.mat4(), glm.vec3(planet_info.radius, planet_info.radius, planet_info.radius))
        
        # calcula a matriz de transformação final da Terra
        self.earthPlanet.modelMatrix = self.globalTransform * transl_planet_pos * scale_planet_size
        
        # atualiza o shader com a transformação e a cor da Terra
        self.shaderProgram.setUniformMVPMatrix(self.earthPlanet.modelMatrix)
        self.shaderProgram.setUniformColor(self.earthPlanet.color)
        
        # renderiza a Terra
        self.earthPlanet.renderer.render()
    
    def renderMoonOrbit(self, time_elapsed):
        
        ##########################################################
        # Adicione aqui o código para renderizar a órbita da lua #
        ##########################################################
        
        pass #remova essa linha quando adicionar o seu código
        
    def renderMoon(self, time_elapsed):
        
        ################################################
        # Adicione aqui o código para renderizar a lua #
        ################################################
            
        pass #remova essa linha quando adicionar o seu código
        
def main():
    import sys

    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    w = MyWidget(glformat)
    w.resize(600, 600)
    w.setWindowTitle('Solar System')
    w.show()
    
    output = app.exec_()
    sys.exit(output)
    
if __name__ == '__main__':
    main()

SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
